In [2]:
import torch.nn as nn

In [62]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=9)
        self.conv_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7)
        self.conv_3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.conv_4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv_5 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3)

        self.dropout_dense = nn.Dropout(p=0.5)
        self.max_pool = nn.MaxPool2d(2)
        self.dropout_pool = nn.Dropout2d(p=0.2)
        
        self.flatten = nn.Flatten(1)
        self.dense_1 = nn.Linear(128, 1024)
        self.dense_2 = nn.Linear(1024, 1024)
        self.output = nn.Linear(1024, 8)
        
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.conv_1(x)
        x = self.max_pool(x)
        x = self.dropout_pool(x)
        x = self.relu(x)
        print("pooling 1",x.shape)

        x = self.conv_2(x)
        x = self.max_pool(x)
        x = self.dropout_pool(x)
        x = self.relu(x)
        print("pooling 2",x.shape)
        

        x = self.conv_3(x)
        x = self.max_pool(x)
        x = self.dropout_pool(x)
        x = self.relu(x)
        print("pooling 3",x.shape)

        x = self.conv_4(x)
        x = self.max_pool(x)
        x = self.dropout_pool(x)
        x = self.relu(x)
        print("pooling 4",x.shape)

        x = self.conv_5(x)
        x = self.max_pool(x)
        x = self.relu(x)
        x = self.dropout_pool(x)
        print("pooling 5",x.shape)

        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.dropout_dense(x)
        
        x = self.dense_2(x)
        x = self.dropout_dense(x)
        
        x = self.output(x)
        return self.softmax(x)


        

In [63]:
inputs = torch.randint(0, 255, (32, 3, 128, 128),dtype=torch.float32)
alex_net = AlexNet()
out = alex_net(inputs)
"output -->", out.shape

pooling 1 torch.Size([32, 16, 60, 60])
pooling 2 torch.Size([32, 32, 27, 27])
pooling 3 torch.Size([32, 64, 11, 11])
pooling 4 torch.Size([32, 128, 4, 4])
pooling 5 torch.Size([32, 128, 1, 1])


('output -->', torch.Size([32, 8]))